<a href="https://colab.research.google.com/github/Minyst/ML_DL_Portfolio_KR/blob/main/Detectron/Detectron2_Custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
!curl -L "https://universe.roboflow.com/ds/vAyJFC7aFj?key=9ddtOvgU5P" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
import os
from detectron2.data.datasets import register_coco_instances

# 사용자 정의 데이터셋 경로
train_json = "/content/train/_annotations.coco.json"
val_json = "/content/valid/_annotations.coco.json"

# 데이터셋 등록
register_coco_instances("my_dataset_train", {}, train_json, "/content/train")
register_coco_instances("my_dataset_val", {}, val_json, "/content/valid")

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo

# Config 설정
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)
cfg.DATALOADER.NUM_WORKERS = 2

# COCO 데이터셋으로 사전 학습된 모델
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
# cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # 학습된 모델의 가중치 불러오기

cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 100
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=True)  # 이전 학습 상태에서 학습을 재개
trainer.train()

In [ ]:
import cv2
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# Config 설정
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # 학습된 모델의 가중치 불러오기
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.4
predictor = DefaultPredictor(cfg)

input_video_path = '/content/PREMIER LEAGUE HIGHLIGHTS 10sec.mp4'

In [ ]:
#한 frame의 shape 확인하기
cap = cv2.VideoCapture(input_video_path)
while cap.isOpened():
    ret, frame = cap.read()
    print(frame.shape)
    if ret:
        break
cap.release()

In [ ]:
# 모든 frame의 shape이랑 개수 확인하기
cap = cv2.VideoCapture(input_video_path)
cnt = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    print(frame.shape)
    cnt+=1
print(cnt)
cap.release()

In [ ]:
#frame을 list에 담기
cap = cv2.VideoCapture(input_video_path)
frames = []
while cap.isOpened():
    ret, frame = cap.read()
    frames.append(frame)
    if not ret:
        break

In [ ]:
#예측한 하나의 frame shape확인하기
for frame in frames:
    outputs = predictor(frame)
    v = Visualizer(frame[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    result = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    result_image = result.get_image()[:, :, ::-1]
    print(result_image.shape)
    break

width1 = result_image.shape[0]
height1 = result_image.shape[1]
print(width1, height1)

In [ ]:
#예측한 모든 frame shape확인하기
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

cnt = 0
result_images_lst = []

for frame in frames:
    if frame is None:
        break
    outputs = predictor(frame)
    v = Visualizer(frame[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    result = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    result_image = result.get_image()[:, :, ::-1]
    print(result_image.shape)
    result_images_lst.append(result_image)
    cnt+=1

width2 = result_image.shape[0]
height2 = result_image.shape[1]
print(width2, height2)
print(cnt)

In [ ]:
cap = cap = cv2.VideoCapture(input_video_path)
fps = cnt/10

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video_path = '/content/output_video.mp4'
out = cv2.VideoWriter(output_video_path, fourcc, fps, (height2, width2))

for result_image in result_images_lst:
    if result_image is None:
        break
    out.write(result_image)

cap.release()
out.release()